Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [22]:
import pandas as pd

# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [23]:
len(train)

4087

In [24]:
train.ratingCategory.value_counts()

1    2881
0    1141
2      65
Name: ratingCategory, dtype: int64

In [25]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [26]:
test.head()

,id,description
0,3461,\nStyle: Speyside single malt scotch Color: Wa...
1,2604,"\nVery bright and lively, with a nice balance ..."
2,3341,\nA new oloroso-forward Chivas positioned to s...
3,3764,\nAged in bourbon casks and then enhanced in R...
4,2306,\nThere is a freshness to the wood on the nose...


In [27]:
import re

def clean_data(text):
    period_commas = "[.,]"
    non_alpha = "[^a-zA-Z]"
    single_chars = r"\b[a-zA-Z]\b"
    multi_whitespace = "[ ]{2,}"    
    
    text = re.sub(period_commas, "", text)
    text = re.sub(non_alpha, " ", text)
    text = re.sub(single_chars, "", text)
    text = re.sub(multi_whitespace, " ", text)

    return text.lower().strip()

In [28]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [29]:
def lemmatize(doc):
    lemmas = []
    for token in nlp(doc):
        if (token.is_digit == False) and (token.is_punct == False) and (token.is_stop == False) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
    return lemmas

In [30]:
def list_to_string(text):
  """
  Converts list of tokens to one long string
  """
  str = " "
  return (str.join(text))

In [31]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=10)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [32]:
train['clean_data'] = train['description'].apply(clean_data)

In [33]:
test['clean_data'] = test['description'].apply(clean_data)

In [34]:
test.head()

,id,description,clean_data
0,3461,\nStyle: Speyside single malt scotch Color: Wa...,style speyside single malt scotch color walnut...
1,2604,"\nVery bright and lively, with a nice balance ...",very bright and lively with nice balance of fl...
2,3341,\nA new oloroso-forward Chivas positioned to s...,new oloroso forward chivas positioned to split...
3,3764,\nAged in bourbon casks and then enhanced in R...,aged in bourbon casks and then enhanced in rio...
4,2306,\nThere is a freshness to the wood on the nose...,there is freshness to the wood on the nose lac...


In [35]:
%%time

train['lemmas_list'] = train['clean_data'].parallel_apply(lemmatize)

CPU times: user 1.62 s, sys: 412 ms, total: 2.03 s
Wall time: 16 s


In [36]:
%%time

train['lemmas_string'] = train['lemmas_list'].parallel_apply(list_to_string)

CPU times: user 411 ms, sys: 192 ms, total: 602 ms
Wall time: 841 ms


In [37]:
train.head()

,id,description,ratingCategory,clean_data,lemmas_list,lemmas_string
0,1321,"\nSometimes, when whisky is batched, a few lef...",1,sometimes when whisky is batched few leftover ...,"[whisky, batch, leftover, barrel, return, ware...",whisky batch leftover barrel return warehouse ...
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0,an uncommon exclusive bottling of year old cas...,"[uncommon, exclusive, bottling, year, old, cas...",uncommon exclusive bottling year old cask stre...
2,655,\nThis release is a port version of Amrut’s In...,1,this release is port version of amrut intermed...,"[release, port, version, amrut, intermediate, ...",release port version amrut intermediate sherry...
3,555,\nThis 41 year old single cask was aged in a s...,1,this year old single cask was aged in sherry b...,"[year, old, single, cask, age, sherry, butt, i...",year old single cask age sherry butt interact ...
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1,quite herbal on the nose with aromas of dried ...,"[herbal, nose, aroma, dry, tarragon, parsley, ...",herbal nose aroma dry tarragon parsley dill ch...


In [38]:
%%time

test['lemmas_list'] = test['clean_data'].parallel_apply(lemmatize)

CPU times: user 713 ms, sys: 231 ms, total: 944 ms
Wall time: 4.62 s


In [39]:
%%time 
test['lemmas_string'] = test['lemmas_list'].parallel_apply(lambda text: " ".join(text))

CPU times: user 387 ms, sys: 205 ms, total: 592 ms
Wall time: 673 ms


In [40]:
test.head()

,id,description,clean_data,lemmas_list,lemmas_string
0,3461,\nStyle: Speyside single malt scotch Color: Wa...,style speyside single malt scotch color walnut...,"[style, speyside, single, malt, scotch, color,...",style speyside single malt scotch color walnut...
1,2604,"\nVery bright and lively, with a nice balance ...",very bright and lively with nice balance of fl...,"[bright, lively, nice, balance, flavor, zesty,...",bright lively nice balance flavor zesty fruit ...
2,3341,\nA new oloroso-forward Chivas positioned to s...,new oloroso forward chivas positioned to split...,"[new, oloroso, forward, chivas, position, spli...",new oloroso forward chivas position split year...
3,3764,\nAged in bourbon casks and then enhanced in R...,aged in bourbon casks and then enhanced in rio...,"[age, bourbon, cask, enhance, rioja, wine, cas...",age bourbon cask enhance rioja wine cask lange...
4,2306,\nThere is a freshness to the wood on the nose...,there is freshness to the wood on the nose lac...,"[freshness, wood, nose, lace, caramel, delicat...",freshness wood nose lace caramel delicate mint...


In [44]:
train['lemmas_string'][0]

'whisky batch leftover barrel return warehouse canadian club recently pull vat acetone granny smith apple fresh cut white cedar showcase long age complex spicy reserve dram ripe strawberry can pear clove pepper faint flower slightly pull oak tannin distinct elegant remarkably vibrant ancient canadian club tired australia'

In [47]:
train['description'][0]

'\nSometimes, when whisky is batched, a few leftover barrels are returned to the warehouse. Canadian Club recently pulled and vatted several of these from the 1970s. Acetone, Granny Smith apples, and fresh-cut white cedar showcase this long age. Complex and spicy, yet reserved, this dram is ripe with strawberries, canned pears, cloves, pepper, and faint flowers, then slightly pulling oak tannins. Distinct, elegant, and remarkably vibrant, this ancient Canadian Club is anything but tired. (Australia only)\xa0A$133'

### Define Pipeline Components

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

vect = TfidfVectorizer()
clf = RandomForestClassifier(random_state=42, n_jobs=-2)

pipe = Pipeline([('vect', vect),
                 ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [46]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (0.01, 0.05, 0.1),
    'vect__max_features': (1000, 5000, 10000),
    'clf__n_estimators': (100, 200),
    'clf__criterion': ('gini', 'entropy'),
    'clf__max_depth': (5,10,15,20),
    
}

grid_search = GridSearchCV(pipe,
                           parameters,
                           cv=5,
                           n_jobs=-2,
                           verbose=1)

grid_search.fit(train['lemmas_string'], train['ratingCategory'])

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf',
                                        RandomForestClassifier(n_jobs=-2,
                                                               random_state=42))]),
             n_jobs=-2,
             param_grid={'clf__criterion': ('gini', 'entropy'),
                         'clf__max_depth': (5, 10, 15, 20),
                         'clf__n_estimators': (100, 200),
                         'vect__max_df': (0.75, 1.0),
                         'vect__max_features': (1000, 5000, 10000),
                         'vect__min_df': (0.01, 0.05, 0.1)},
             verbose=1)

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [55]:
# Predictions on test sample
pred = grid_search.predict(test['lemmas_string'])

In [56]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [57]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [62]:
submission.ratingCategory.value_counts()

1    968
0     54
Name: ratingCategory, dtype: int64

In [59]:
subNumber = 0

In [60]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 80% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [66]:
from sklearn.decomposition import TruncatedSVD

lsi = TruncatedSVD()
vect = TfidfVectorizer(stop_words='english')
clf = RandomForestClassifier(random_state=42, n_jobs=-2)

lsi = Pipeline([('vect', vect),
                ('svd', lsi)])

pipe = Pipeline([('lsi', lsi),
                 ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [68]:
parameters = {
    'lsi__vect__max_df': (0.75, 1.0),
    'lsi__vect__min_df': (0.01, 0.05, 0.1),
    'lsi__vect__max_features': (1000, 5000, 10000),
    'lsi__svd__n_components': [10, 100],
    'lsi__svd__algorithm': ['arpack', 'randomized'],
    'clf__max_depth':(5,10),
    'clf__n_estimators': (100, 200),
    'clf__criterion': ('gini', 'entropy')
}

grid_search = GridSearchCV(pipe,parameters,
                           cv=5,
                           n_jobs=-2,
                           verbose=1)

grid_search.fit(train['lemmas_string'], train['ratingCategory'])

Fitting 5 folds for each of 576 candidates, totalling 2880 fits


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:925: UserWarning: One or more of the test scores are non-finite: [0.72277968 0.71641763 0.7068747  0.72277968 0.71641763 0.7068747
 0.72277968 0.71641763 0.7068747  0.72277968 0.71641763 0.7068747
 0.72277968 0.71641763 0.7068747  0.72277968 0.71641763 0.7068747
 0.70761029 0.7058976         nan 0.70761029 0.7058976         nan
 0.70761029 0.7058976         nan 0.70761029 0.7058976         nan
 0.70761029 0.7058976         nan 0.70761029 0.7058976         nan
 0.72375948 0.71617463 0.7071183  0.7203341  0.71886441 0.71103117
 0.72106819 0.71886501 0.71005527 0.7203326  0.71959911 0.70883009
 0.7205783  0.71641853 0.70981018 0.72229099 0.71396965 0.70858559
 0.70736579 0.7054074         nan 0.70736549 0.7056528         nan
 0.70687709 0.70467391        nan 0.70663199 0.7056537         nan
 0.7051629  0.7056528         nan 0.70663109 0.7051635         nan
 0.72204499 0.71739712 0.70785209 0.72204499 0.71739712 0.70

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('lsi',
                                        Pipeline(steps=[('vect',
                                                         TfidfVectorizer(stop_words='english')),
                                                        ('svd',
                                                         TruncatedSVD())])),
                                       ('clf',
                                        RandomForestClassifier(n_jobs=-2,
                                                               random_state=42))]),
             n_jobs=-2,
             param_grid={'clf__criterion': ('gini', 'entropy'),
                         'clf__max_depth': (5, 10),
                         'clf__n_estimators': (100, 200),
                         'lsi__svd__algorithm': ['arpack', 'randomized'],
                         'lsi__svd__n_components': [10, 100],
                         'lsi__vect__max_df': (0.75, 1.0),
                         'lsi__ve

### Make a Submission File

In [69]:
# Predictions on test sample
pred = grid_search.predict(test['lemmas_string'])

In [70]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [71]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [72]:
submission.ratingCategory.value_counts()

1    902
0    120
Name: ratingCategory, dtype: int64

In [73]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [42]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [43]:
X_train = train['lemmas_string'].values
X_test = test['lemmas_string'].values

In [44]:
X_train_vect = get_word_vectors(X_train)
X_test_vect = get_word_vectors(X_test)

In [45]:
Y_train = train['ratingCategory'].values

In [48]:
# Apply to your Dataset

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
# from scipy.stats import randint

gbc = GradientBoostingClassifier(random_state=42)

pipe = Pipeline([('clf', gbc)])

param_dist = {
    'clf__n_estimators': (100, 500, 1000),
    'clf__loss': ('deviance', 'exponential'),
    'clf__max_depth' : (3, 5, 10),
    'clf__min_samples_leaf': (2,7,15),
    'clf__criterion': ('friedman_mse', 'mse', 'mae')
}

rs = GridSearchCV(pipe,
                param_dist,
                cv=5,
                n_jobs=-2,
                verbose=1)

rs.fit(X_train_vect, Y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


KeyboardInterrupt: 

In [ ]:
print("The function finally ran :D")

In [ ]:
# Apply to your Dataset
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint
from sklearn.pipeline import make_pipeline


# gbc = GradientBoostingClassifier(random_state=42)

pipe = make_pipeline(GradientBoostingClassifier(random_state=42))

param_dist = {
    'gradientboostingclassifier__n_estimators': (100, 500, 1000),
    'gradientboostingclassifier__loss': ('deviance', 'exponential'),
    'gradientboostingclassifier__max_depth' : (3, 5, 10),
    'gradientboostingclassifier__min_samples_leaf': (2,7,15),
    'gradientboostingclassifier__criterion': ('friedman_mse', 'mse', 'mae')
}

gs = GridSearchCV(pipe,
                param_dist,
                cv=5,
                n_jobs=-2,
                verbose=1)

gs.fit(X_train_vect, Y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


### Make a Submission File

In [ ]:
# Predictions on test sample
pred = gs.predict(X_test_vect)

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
submission.ratingCategory.value_counts()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 80% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?